# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from psycopg2.errors import UniqueViolation
import json
import datetime
import re
import sql_queries

In [6]:
conn.close()

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
cur.execute(sql_queries.song_table_create)

In [5]:
conn.commit()

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('./data/song_data')

In [5]:
len(song_files)

71

In [6]:
log_files = get_files('./data/log_data/2018/11')

In [7]:
len(log_files)

30

In [8]:
log_files[0]

'/Users/markbrennan/udacity/projects/data_modeling_with_postgres/data/log_data/2018/11/2018-11-11-events.json'

In [9]:
df_songs = pd.read_json(song_files[0], lines=True)
df_songs.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR7G5I41187FB4CE6C,NaN,"London, England",NaN,Adam Ant,233.40363,1,SONHOTT12A8C13493C,Something Girls,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
song_data = 
song_data

In [12]:
df_songs.values[0][0]

'AR7G5I41187FB4CE6C'

In [13]:
df_songs.artist_id

0    AR7G5I41187FB4CE6C
Name: artist_id, dtype: object

In [22]:
idx=0

In [32]:
song_data = (df_songs.song_id[idx], 
             df_songs.title[idx], 
             df_songs.artist_id[idx], 
             int(df_songs.year[idx]), 
             float(df_songs.duration[idx]))

In [70]:
song_table_insert = """
INSERT INTO songs(song_id, title, artist_id, year, duration)
VALUES (%s, %s, %s, %s, %s)
"""

In [88]:
for file in song_files_test:
    df_songs = pd.read_json(file, lines=True)
    for idx, row in df_songs.iterrows():
        song_data = (row.song_id, 
                     row.title, 
                     row.artist_id, 
                     int(row.year), 
                     float(row.duration))
        print(f'inserting {song_data}')
        try:
            cur.execute(song_table_insert, song_data)
        except UniqueViolation as e:
                print('caught UniqueViolation exception!')
                print(e.pgerror)
                print(e.diag.message_primary)
                continue

inserting ('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, 233.40363)
inserting ('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, 269.81832)


## READ AND INSERT SONG DATA

In [12]:
def read_and_insert_song_data(song_files):
    song_table_insert = """
    INSERT INTO songs(song_id, title, artist_id, year, duration)
    VALUES (%s, %s, %s, %s, %s)
    """    
    for file in song_files:
        with open(file) as f:
            try:
                data = json.load(f)
                song_data = (data['song_id'], 
                             data['title'],
                             data['artist_id'],
                             data['year'],
                             data['duration'])
                print(f'inserting {song_data}')

                cur.execute(song_table_insert, song_data)
            except psycopg2.Error as e:
                    print('caught psycopg2 exception!')
                    print(e.pgerror)
                    print(e.diag.message_primary)
                    continue
            except KeyError as e:
                print(f'Key Error:  {str(e)}')
                continue   
            except JSONDecodeError as e:
                print('Msg: {e.msg}, Doc: {e.doc}, Pos: {e.pos}, LineNo: {e.lineno}, ColNo: {e.colno}')
                continue   
    conn.commit()

In [15]:
def get_song_and_artist_data(song_files):
    song_data = []
    artist_data = []
    for file in song_files:
        with open(file) as f:
            try:
                data = json.load(f)
                song_data.append(dict(song_id=data['song_id'],
                                      title=data['title'],
                                      artist_id=data['artist_id'],
                                      year=data['year'],
                                      duration=data['duration']))
                artist_data.append(dict(artist_id=data['artist_id'],
                                        artist_name=data['artist_name'],
                                        artist_location=data['artist_location'],
                                        artist_latitude=data['artist_latitude'],
                                        artist_longitude=data['artist_longitude']))

            except KeyError as e:
                print(f'Key Error:  {str(e)}')
                continue   
            except JSONDecodeError as e:
                print('Msg: {e.msg}, Doc: {e.doc}, Pos: {e.pos}, LineNo: {e.lineno}, ColNo: {e.colno}')
                continue
    return song_data, artist_data

In [16]:
song_data, artist_data = get_song_and_artist_data(song_files)

In [22]:
def insert_song_data(song_data):
    song_table_insert = """
                        INSERT INTO songs(song_id, title, artist_id, year, duration)
                        VALUES (%s, %s, %s, %s, %s)
                        """    
    for song in song_data:
        try:
            song_data = (song['song_id'], 
                         song['title'],
                         song['artist_id'],
                         song['year'],
                         song['duration'])
            print(f'inserting {song_data}')
            cur.execute(song_table_insert, song_data)
        except psycopg2.Error as e:
            print('caught psycopg2 exception!')
            print(e.pgerror)
            print(e.diag.message_primary)
            continue
        except KeyError as e:
            print(f'Key Error:  {str(e)}')
            continue   
            
    conn.commit()    

In [23]:
insert_song_data(song_data)

inserting ('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', 1982, 233.40363)
inserting ('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, 269.81832)
inserting ('SOFSOCN12A8C143F5D', 'Face the Ashes', 'ARXR32B1187FB57099', 2007, 209.60608)
inserting ('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, 189.57016)
inserting ('SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B', 0, 218.77506)
inserting ('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546)
inserting ('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179)
inserting ('SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0, 177.47546)
inserting ('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', 0, 266.39628)
inserting ('SOYMRWW12A6D4FAB14', 'The Moon And I (Ordinary Day Album Version)', 'ARKFYS91187B98E58F', 0, 267.7024)
inserting ('SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5

In [28]:
def insert_artist_data(artist_data):
    artist_table_insert = """INSERT INTO artists(artist_id, name, location, latitude, longitude) 
                                VALUES(%s, %s, %s, %s, %s)"""
    artists_seen = set()    
    for artist in artist_data:
        artist_id = artist['artist_id']
        if artist_id and artist_id not in artists_seen:
            artists_seen.add(artist_id)
            artist_data = (artist_id, 
                           artist['artist_name'],
                           artist['artist_location'],
                           artist['artist_latitude'],
                           artist['artist_longitude'])
            print(f'inserting {artist_data}')
            try:
                cur.execute(artist_table_insert, artist_data)
            except psycopg2.Error as e:
                    print('caught psycopg2 exception!')
                    print(e.pgerror)
                    print(e.diag.message_primary)
                    continue
    conn.commit()

In [29]:
insert_artist_data(artist_data)

inserting ('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', None, None)
inserting ('AR8ZCNI1187B9A069B', 'Planet P Project', '', None, None)
inserting ('ARXR32B1187FB57099', 'Gob', '', None, None)
inserting ('AR10USD1187B99F3F1', 'Tweeterfriendly Music', 'Burlington, Ontario, Canada', None, None)
inserting ('ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962, -80.01955)
inserting ('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892)
inserting ('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None)
inserting ('ARKRRTF1187B9984DA', 'Sonora Santanera', '', None, None)
inserting ('ARNTLGG11E2835DDB9', 'Clp', '', None, None)
inserting ('ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', None, None)
inserting ('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', None, None)
inserting ('ARH4Z031187B9A71F2', 'Faye Adams', 'Newark, NJ', 40.73197, -74.17418)
inserting ('ARB29H41187B98F0EF', 'Terry Callier', 'Chicago', 41.88415, -87.63241)
inserting ('AR0RCMP1187F

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [30]:
log_files = get_files('./data/log_data/2018/11')

In [31]:
log_files[0]

'/Users/markbrennan/udacity/projects/data_modeling_with_postgres/data/log_data/2018/11/2018-11-11-events.json'

In [32]:
len(log_files)

30

In [ ]:
filepath = 

In [33]:
df_log = pd.read_json(log_files[0], lines=True)
df_log.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


In [ ]:
#timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and

In [ ]:
# user_id', 'first', 'last', 'gender','level' 

## DO THE MAIN WORK OF READING LOG/EVENT DATA

In [34]:
def get_all_log_data(log_files):
    all_log_data = []
    for file in log_files:
        with open(file, 'r') as f:
            for line in f:
                try:
                    data = json.loads(line)
                except JSONDecodeError as e:
                    print('Msg: {e.msg}, Doc: {e.doc}, Pos: {e.pos}, LineNo: {e.lineno}, ColNo: {e.colno}')
                    continue
                    
                if data['page'] == 'NextSong':
                    try:
                        all_log_data.append(dict(ts=data['ts'],
                                                 user_id=data['userId'],
                                                 first_name=data['firstName'],
                                                 last_name=data['lastName'],
                                                 gender=data['gender'],
                                                 level=data['level'],
                                                 song_title=data['song'],
                                                 artist_name=data['artist'],
                                                 length=data['length'],
                                                 session_id=data['sessionId'],
                                                 location=data['location'],
                                                 user_agent=data['userAgent']
                                                ))
                    except KeyError as e:
                        print(f'Key Error:  {str(e)}')
                        continue
    return all_log_data

In [35]:
all_log_data = get_all_log_data(log_files)

In [36]:
all_log_data[0]

{'ts': 1541903636796,
 'user_id': '69',
 'first_name': 'Anabelle',
 'last_name': 'Simpson',
 'gender': 'F',
 'level': 'free',
 'song_title': 'Fuck Kitty',
 'artist_name': 'Frumpies',
 'length': 134.47791,
 'session_id': 455,
 'location': 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
 'user_agent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'}

In [37]:
len(all_log_data)

6820

## TIME DATA

In [38]:
INSERT_TIME_QUERY = """INSERT INTO time(timestamp, hour, day, week, month, year, weekday) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s)"""

In [53]:
def get_timestamp(ts):
    dt = datetime.datetime.fromtimestamp(ts / 1000)
    return dt, dt.strftime('%Y-%m-%d %H:%M:%S.%f')

In [57]:
def insert_time_data(all_log_data, time_query):
    seen_timestamps = set()
    for entry in all_log_data:
        ts = None
        try:
            ts = entry['ts']
        except KeyError as e:
            print(f'Key Error:  {str(e)}')
            continue
        if ts and ts not in seen_timestamps:
            seen_timestamps.add(ts)
            dt, timestamp = get_timestamp(ts)
            hour = dt.hour
            day = dt.day
            year, week, weekday = dt.isocalendar()
            month = dt.month
            insert_vals = (timestamp, hour, day, week, month, year, weekday)
            try:
                cur.execute(time_query, insert_vals)
            except psycopg2.Error as e:
                    print('caught psycopg2 exception!')
                    print(e.pgerror)
                    print(e.diag.message_primary)
                    continue
    conn.commit()

In [58]:
insert_time_data(all_log_data, INSERT_TIME_QUERY)

## USER DATA

In [62]:
INSERT_USER_QUERY = """INSERT INTO users(user_id, first_name, last_name, gender, level) 
                        VALUES (%s, %s, %s, %s, %s)"""

In [65]:
def insert_user_data(all_log_data, insert_user_query):
    seen_users = set()
    for entry in all_log_data:
        user_id = first_name = last_name = gender = level = None
        try:
            user_id = entry['user_id']
            first_name = entry['first_name']
            last_name = entry['last_name']
            gender = entry['gender']
            level = entry['level']
        except KeyError as e:
            print(f'Key Error:  {str(e)}')
            continue
        if user_id and user_id not in seen_users and first_name and last_name and gender and level: 
            seen_users.add(user_id)
            insert_vals = (user_id, first_name, last_name, gender, level)
            try:
                cur.execute(insert_user_query, insert_vals)
            except psycopg2.Error as e:
                    print('caught psycopg2 exception!')
                    print(e.pgerror)
                    print(e.diag.message_primary)
                    continue    
    conn.commit()

In [66]:
insert_user_data(all_log_data, INSERT_USER_QUERY)

## SONG PLAYS DATA

In [337]:
SONG_ARTIST_ID_QRY = "SELECT song_id, s.artist_id \
FROM songs s JOIN artists a on s.artist_id = a.artist_id \
WHERE title = '{}' and a.name='{}' and duration={}"

In [338]:
SONG_ARTIST_ID_QRY

"SELECT song_id, s.artist_id FROM songs s JOIN artists a on s.artist_id = a.artist_id WHERE title = '{}' and a.name='{}' and duration={}"

In [339]:
SONG_ARTIST_ID_QRY.format(title,name,duration)

"SELECT song_id, s.artist_id FROM songs s JOIN artists a on s.artist_id = a.artist_id WHERE title = 'Setanta matins' and a.name='Elena' and duration=269.58322"

Implement the song_select query in sql_queries.py to find the song ID and artist ID based on the title, artist name, and duration of a song.
Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to songplay_data

In [333]:
df_log.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Great Lake Swimmers,Logged In,Kevin,M,0,Arellano,215.11791,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Your Rocky Spine,200,1542931645796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


In [344]:
all_log_data[0]

{'ts': 1541903636796,
 'user_id': '69',
 'first_name': 'Anabelle',
 'last_name': 'Simpson',
 'gender': 'F',
 'level': 'free',
 'song_title': 'Fuck Kitty',
 'artist_name': 'Frumpies',
 'length': 134.47791,
 'session_id': 455,
 'location': 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
 'user_agent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"'}

In [347]:
#songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

In [68]:
def insert_song_play_data(all_log_data):
    song_artist_id_qry = "SELECT song_id, s.artist_id \
                            FROM songs s JOIN artists a on s.artist_id = a.artist_id \
                            WHERE title = '{}' and a.name = '{}' and duration = {}"
    insert_query = """INSERT INTO songplays(start_time, user_id, level, song_id, artist_id, session_id, location, user_agent) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
    for entry in all_log_data:
        # find the song ID and artist ID based on the title, artist name, and duration of a song.
        #  timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent 
        song_title = artist_name = duration = None
        song_title = entry['song_title'] 
        artist_name = entry['artist_name']
        duration = entry['length']
        if song_title and artist_name and duration:
            # escape apostrophes in song titles
            song_title = re.sub("'", "''", song_title)
            # and escape apostrophes in artist names
            artist_name = re.sub("'", "''", artist_name)            
            query = song_artist_id_qry.format(song_title, artist_name, duration)
            try:
                song_id = artist_id = None
                cur.execute(query)
                row = cur.fetchone()
                if row:
                    song_id, artist_id = row
                
                _, timestamp = get_timestamp(entry['ts'])
                user_id = entry['user_id']
                level = entry['level']
                session_id = entry['session_id']
                location = entry['location']
                user_agent = entry['user_agent']
                insert_vals = (timestamp, user_id, level, song_id, artist_id, 
                               session_id, location, duration)
                cur.execute(insert_query, insert_vals)

            except psycopg2.Error as e:
                print('caught psycopg2 exception!')
                print(e.pgerror)
                print(e.diag.message_primary)
                continue     
            except KeyError as e:
                print(f'Key Error:  {str(e)}')
                continue
        conn.commit()

In [69]:
insert_song_play_data(all_log_data)

In [445]:
qry = """SELECT song_id, s.artist_id                             FROM songs s JOIN artists a on s.artist_id = a.artist_id                             
        WHERE title = 'Setanta matins' and a.name = 'Elena' and duration = 269.58322"""

In [441]:
cur.execute(qry)

In [442]:
print(cur.fetchone())

None


In [439]:
if row:
    song_id, artist_id = row
    print(song_id, artist_id)

SOZCTXZ12AB0182364 AR5KOSW1187FB35FF4


## OLDER QUERIES - Keep as notes, but do not use

In [217]:
# #start_time, hour, day, week, month, year, weekday
# seen_timestamps = set()
# for file in log_files:
#     with open(file, 'r') as f:
#         for line in f:
#             d = json.loads(line)
#             ts = d['ts']
#             print(ts)
#             if ts not in seen_timestamps:
#                 seen_timestamps.add(ts)
#                 dt = datetime.fromtimestamp(ts / 1000)
#                 timestamp = dt.strftime('%Y-%m-%d %H:%M:%S.%f')
#                 hour = dt.hour
#                 day = dt.day
#                 year, week, weekday = dt.isocalendar()
#                 month = dt.month
#                 insert_vals = (timestamp, hour, day, week, month, year, weekday)
#                 print(f'insert_vals: {insert_vals}')
#                 try:
#                     cur.execute(time_query, insert_vals)
#                 except psycopg2.Error as e:
#                         print('caught psycopg2 exception!')
#                         print(e.pgerror)
#                         print(e.diag.message_primary)
#                         continue

1541903636796
insert_vals: ('2018-11-10 21:33:56.796000', 21, 10, 45, 11, 2018, 6)
1541903770796
insert_vals: ('2018-11-10 21:36:10.796000', 21, 10, 45, 11, 2018, 6)
1541904034796
insert_vals: ('2018-11-10 21:40:34.796000', 21, 10, 45, 11, 2018, 6)
1541910841796
insert_vals: ('2018-11-10 23:34:01.796000', 23, 10, 45, 11, 2018, 6)
1541910973796
insert_vals: ('2018-11-10 23:36:13.796000', 23, 10, 45, 11, 2018, 6)
1541911006796
insert_vals: ('2018-11-10 23:36:46.796000', 23, 10, 45, 11, 2018, 6)
1541930188796
insert_vals: ('2018-11-11 04:56:28.796000', 4, 11, 45, 11, 2018, 7)
1541930447796
insert_vals: ('2018-11-11 05:00:47.796000', 5, 11, 45, 11, 2018, 7)
1541930647796
insert_vals: ('2018-11-11 05:04:07.796000', 5, 11, 45, 11, 2018, 7)
1541930908796
insert_vals: ('2018-11-11 05:08:28.796000', 5, 11, 45, 11, 2018, 7)
1541931120796
insert_vals: ('2018-11-11 05:12:00.796000', 5, 11, 45, 11, 2018, 7)
1541932052796
insert_vals: ('2018-11-11 05:27:32.796000', 5, 11, 45, 11, 2018, 7)
1541932063

1542992337796
insert_vals: ('2018-11-23 11:58:57.796000', 11, 23, 47, 11, 2018, 5)
1542992342796
insert_vals: ('2018-11-23 11:59:02.796000', 11, 23, 47, 11, 2018, 5)
1542992455796
insert_vals: ('2018-11-23 12:00:55.796000', 12, 23, 47, 11, 2018, 5)
1542992500796
insert_vals: ('2018-11-23 12:01:40.796000', 12, 23, 47, 11, 2018, 5)
1542992519796
insert_vals: ('2018-11-23 12:01:59.796000', 12, 23, 47, 11, 2018, 5)
1542992707796
insert_vals: ('2018-11-23 12:05:07.796000', 12, 23, 47, 11, 2018, 5)
1542992907796
insert_vals: ('2018-11-23 12:08:27.796000', 12, 23, 47, 11, 2018, 5)
1542992917796
insert_vals: ('2018-11-23 12:08:37.796000', 12, 23, 47, 11, 2018, 5)
1542992997796
insert_vals: ('2018-11-23 12:09:57.796000', 12, 23, 47, 11, 2018, 5)
1542993192796
insert_vals: ('2018-11-23 12:13:12.796000', 12, 23, 47, 11, 2018, 5)
1542993212796
insert_vals: ('2018-11-23 12:13:32.796000', 12, 23, 47, 11, 2018, 5)
1542993231796
insert_vals: ('2018-11-23 12:13:51.796000', 12, 23, 47, 11, 2018, 5)
1542

1541324292796
insert_vals: ('2018-11-04 04:38:12.796000', 4, 4, 44, 11, 2018, 7)
1541324515796
insert_vals: ('2018-11-04 04:41:55.796000', 4, 4, 44, 11, 2018, 7)
1541324743796
insert_vals: ('2018-11-04 04:45:43.796000', 4, 4, 44, 11, 2018, 7)
1541324956796
insert_vals: ('2018-11-04 04:49:16.796000', 4, 4, 44, 11, 2018, 7)
1541325032796
insert_vals: ('2018-11-04 04:50:32.796000', 4, 4, 44, 11, 2018, 7)
1541325083796
insert_vals: ('2018-11-04 04:51:23.796000', 4, 4, 44, 11, 2018, 7)
1541325328796
insert_vals: ('2018-11-04 04:55:28.796000', 4, 4, 44, 11, 2018, 7)
1541325331796
insert_vals: ('2018-11-04 04:55:31.796000', 4, 4, 44, 11, 2018, 7)
1541325506796
insert_vals: ('2018-11-04 04:58:26.796000', 4, 4, 44, 11, 2018, 7)
1541325548796
insert_vals: ('2018-11-04 04:59:08.796000', 4, 4, 44, 11, 2018, 7)
1541325701796
insert_vals: ('2018-11-04 05:01:41.796000', 5, 4, 44, 11, 2018, 7)
1541325753796
insert_vals: ('2018-11-04 05:02:33.796000', 5, 4, 44, 11, 2018, 7)
1541325860796
insert_vals: (

1542189478796
insert_vals: ('2018-11-14 04:57:58.796000', 4, 14, 46, 11, 2018, 3)
1542189606796
insert_vals: ('2018-11-14 05:00:06.796000', 5, 14, 46, 11, 2018, 3)
1542189677796
insert_vals: ('2018-11-14 05:01:17.796000', 5, 14, 46, 11, 2018, 3)
1542189907796
insert_vals: ('2018-11-14 05:05:07.796000', 5, 14, 46, 11, 2018, 3)
1542190231796
insert_vals: ('2018-11-14 05:10:31.796000', 5, 14, 46, 11, 2018, 3)
1542190394796
insert_vals: ('2018-11-14 05:13:14.796000', 5, 14, 46, 11, 2018, 3)
1542190583796
insert_vals: ('2018-11-14 05:16:23.796000', 5, 14, 46, 11, 2018, 3)
1542190865796
insert_vals: ('2018-11-14 05:21:05.796000', 5, 14, 46, 11, 2018, 3)
1542191238796
insert_vals: ('2018-11-14 05:27:18.796000', 5, 14, 46, 11, 2018, 3)
1542191429796
insert_vals: ('2018-11-14 05:30:29.796000', 5, 14, 46, 11, 2018, 3)
1542191697796
insert_vals: ('2018-11-14 05:34:57.796000', 5, 14, 46, 11, 2018, 3)
1542191872796
insert_vals: ('2018-11-14 05:37:52.796000', 5, 14, 46, 11, 2018, 3)
1542192108796
in

1542213730796
insert_vals: ('2018-11-14 11:42:10.796000', 11, 14, 46, 11, 2018, 3)
1542213742796
insert_vals: ('2018-11-14 11:42:22.796000', 11, 14, 46, 11, 2018, 3)
1542213899796
insert_vals: ('2018-11-14 11:44:59.796000', 11, 14, 46, 11, 2018, 3)
1542214271796
insert_vals: ('2018-11-14 11:51:11.796000', 11, 14, 46, 11, 2018, 3)
1542214537796
insert_vals: ('2018-11-14 11:55:37.796000', 11, 14, 46, 11, 2018, 3)
1542214836796
insert_vals: ('2018-11-14 12:00:36.796000', 12, 14, 46, 11, 2018, 3)
1542215058796
insert_vals: ('2018-11-14 12:04:18.796000', 12, 14, 46, 11, 2018, 3)
1542215370796
insert_vals: ('2018-11-14 12:09:30.796000', 12, 14, 46, 11, 2018, 3)
1542215614796
insert_vals: ('2018-11-14 12:13:34.796000', 12, 14, 46, 11, 2018, 3)
1542215827796
insert_vals: ('2018-11-14 12:17:07.796000', 12, 14, 46, 11, 2018, 3)
1542216131796
insert_vals: ('2018-11-14 12:22:11.796000', 12, 14, 46, 11, 2018, 3)
1542216442796
insert_vals: ('2018-11-14 12:27:22.796000', 12, 14, 46, 11, 2018, 3)
1542

...

1542837407796
insert_vals: ('2018-11-21 16:56:47.796000', 16, 21, 47, 11, 2018, 3)
1542837676796
insert_vals: ('2018-11-21 17:01:16.796000', 17, 21, 47, 11, 2018, 3)
1542837934796
insert_vals: ('2018-11-21 17:05:34.796000', 17, 21, 47, 11, 2018, 3)
1542838144796
insert_vals: ('2018-11-21 17:09:04.796000', 17, 21, 47, 11, 2018, 3)
1542838276796
insert_vals: ('2018-11-21 17:11:16.796000', 17, 21, 47, 11, 2018, 3)
1542838466796
insert_vals: ('2018-11-21 17:14:26.796000', 17, 21, 47, 11, 2018, 3)
1542838743796
insert_vals: ('2018-11-21 17:19:03.796000', 17, 21, 47, 11, 2018, 3)
1542838991796
insert_vals: ('2018-11-21 17:23:11.796000', 17, 21, 47, 11, 2018, 3)
1542839217796
insert_vals: ('2018-11-21 17:26:57.796000', 17, 21, 47, 11, 2018, 3)
1542839453796
insert_vals: ('2018-11-21 17:30:53.796000', 17, 21, 47, 11, 2018, 3)
1542839676796
insert_vals: ('2018-11-21 17:34:36.796000', 17, 21, 47, 11, 2018, 3)
1542839888796
insert_vals: ('2018-11-21 17:38:08.796000', 17, 21, 47, 11, 2018, 3)
1542

1543420590796
insert_vals: ('2018-11-28 10:56:30.796000', 10, 28, 48, 11, 2018, 3)
1543420593796
insert_vals: ('2018-11-28 10:56:33.796000', 10, 28, 48, 11, 2018, 3)
1543420724796
insert_vals: ('2018-11-28 10:58:44.796000', 10, 28, 48, 11, 2018, 3)
1543420799796
insert_vals: ('2018-11-28 10:59:59.796000', 10, 28, 48, 11, 2018, 3)
1543420804796
insert_vals: ('2018-11-28 11:00:04.796000', 11, 28, 48, 11, 2018, 3)
1543420856796
insert_vals: ('2018-11-28 11:00:56.796000', 11, 28, 48, 11, 2018, 3)
1543420997796
insert_vals: ('2018-11-28 11:03:17.796000', 11, 28, 48, 11, 2018, 3)
1543421003796
insert_vals: ('2018-11-28 11:03:23.796000', 11, 28, 48, 11, 2018, 3)
1543421098796
insert_vals: ('2018-11-28 11:04:58.796000', 11, 28, 48, 11, 2018, 3)
1543421169796
insert_vals: ('2018-11-28 11:06:09.796000', 11, 28, 48, 11, 2018, 3)
1543421178796
insert_vals: ('2018-11-28 11:06:18.796000', 11, 28, 48, 11, 2018, 3)
1543421426796
insert_vals: ('2018-11-28 11:10:26.796000', 11, 28, 48, 11, 2018, 3)
1543

1542347789796
insert_vals: ('2018-11-16 00:56:29.796000', 0, 16, 46, 11, 2018, 5)
1542351661796
insert_vals: ('2018-11-16 02:01:01.796000', 2, 16, 46, 11, 2018, 5)
1542351662796
insert_vals: ('2018-11-16 02:01:02.796000', 2, 16, 46, 11, 2018, 5)
1542352827796
insert_vals: ('2018-11-16 02:20:27.796000', 2, 16, 46, 11, 2018, 5)
1542355283796
insert_vals: ('2018-11-16 03:01:23.796000', 3, 16, 46, 11, 2018, 5)
1542355376796
insert_vals: ('2018-11-16 03:02:56.796000', 3, 16, 46, 11, 2018, 5)
1542355624796
insert_vals: ('2018-11-16 03:07:04.796000', 3, 16, 46, 11, 2018, 5)
1542355625796
insert_vals: ('2018-11-16 03:07:05.796000', 3, 16, 46, 11, 2018, 5)
1542355762796
insert_vals: ('2018-11-16 03:09:22.796000', 3, 16, 46, 11, 2018, 5)
1542355912796
insert_vals: ('2018-11-16 03:11:52.796000', 3, 16, 46, 11, 2018, 5)
1542355949796
insert_vals: ('2018-11-16 03:12:29.796000', 3, 16, 46, 11, 2018, 5)
1542355979796
insert_vals: ('2018-11-16 03:12:59.796000', 3, 16, 46, 11, 2018, 5)
1542356161796
in

1543038621796
insert_vals: ('2018-11-24 00:50:21.796000', 0, 24, 47, 11, 2018, 6)
1543039092796
insert_vals: ('2018-11-24 00:58:12.796000', 0, 24, 47, 11, 2018, 6)
1543039148796
insert_vals: ('2018-11-24 00:59:08.796000', 0, 24, 47, 11, 2018, 6)
1543039149796
insert_vals: ('2018-11-24 00:59:09.796000', 0, 24, 47, 11, 2018, 6)
1543039183796
insert_vals: ('2018-11-24 00:59:43.796000', 0, 24, 47, 11, 2018, 6)
1543039440796
insert_vals: ('2018-11-24 01:04:00.796000', 1, 24, 47, 11, 2018, 6)
1543039480796
insert_vals: ('2018-11-24 01:04:40.796000', 1, 24, 47, 11, 2018, 6)
1543039598796
insert_vals: ('2018-11-24 01:06:38.796000', 1, 24, 47, 11, 2018, 6)
1543039715796
insert_vals: ('2018-11-24 01:08:35.796000', 1, 24, 47, 11, 2018, 6)
1543039756796
insert_vals: ('2018-11-24 01:09:16.796000', 1, 24, 47, 11, 2018, 6)
1543039899796
insert_vals: ('2018-11-24 01:11:39.796000', 1, 24, 47, 11, 2018, 6)
1543039907796
insert_vals: ('2018-11-24 01:11:47.796000', 1, 24, 47, 11, 2018, 6)
1543040029796
in

1543086055796
insert_vals: ('2018-11-24 14:00:55.796000', 14, 24, 47, 11, 2018, 6)
1543086121796
insert_vals: ('2018-11-24 14:02:01.796000', 14, 24, 47, 11, 2018, 6)
1543086185796
insert_vals: ('2018-11-24 14:03:05.796000', 14, 24, 47, 11, 2018, 6)
1543086275796
insert_vals: ('2018-11-24 14:04:35.796000', 14, 24, 47, 11, 2018, 6)
1543086384796
insert_vals: ('2018-11-24 14:06:24.796000', 14, 24, 47, 11, 2018, 6)
1543086424796
insert_vals: ('2018-11-24 14:07:04.796000', 14, 24, 47, 11, 2018, 6)
1543086642796
insert_vals: ('2018-11-24 14:10:42.796000', 14, 24, 47, 11, 2018, 6)
1543092145796
insert_vals: ('2018-11-24 15:42:25.796000', 15, 24, 47, 11, 2018, 6)
1543092353796
insert_vals: ('2018-11-24 15:45:53.796000', 15, 24, 47, 11, 2018, 6)
1543092558796
insert_vals: ('2018-11-24 15:49:18.796000', 15, 24, 47, 11, 2018, 6)
1543097750796
insert_vals: ('2018-11-24 17:15:50.796000', 17, 24, 47, 11, 2018, 6)
1543097978796
insert_vals: ('2018-11-24 17:19:38.796000', 17, 24, 47, 11, 2018, 6)
1543

In [316]:
# with open(log_files[0], 'r') as f:
#     for line in f:
#         d = json.loads(line)
#         print(d)
#         break

In [317]:
# from json import JSONDecodeError
# def read_log_data(log_files):
#     timestamps = []
#     users = []
#     for file in log_files:
#         with open(file, 'r') as f:
#             for line in f:
#                 try:
#                     d = json.loads(line)
#                 except JSONDecodeError as e:
#                     print('Msg: {e.msg}, Doc: {e.doc}, Pos: {e.pos}, LineNo: {e.lineno}, ColNo: {e.colno}')
#                     continue
                    
#                 if d['page'] == 'NextSong':
#                     timestamps.append(d['ts'])
#                     # user ID, first name, last name, gender and level
#                     users.append(dict(user_id=d['userId'],
#                                       first=d['firstName'],
#                                       last=d['lastName'],
#                                       gender=d['gender'],
#                                       level=d['level']))
#     return timestamps, users

In [226]:
# timestamps, users = read_log_data(log_files)

In [318]:
# def insert_time_data(timestamps):
#     seen_timestamps = set()
#     for ts in timestamps:
#         if ts not in seen_timestamps:
#             seen_timestamps.add(ts)
#             dt = datetime.fromtimestamp(ts / 1000)
#             timestamp = dt.strftime('%Y-%m-%d %H:%M:%S.%f')
#             hour = dt.hour
#             day = dt.day
#             year, week, weekday = dt.isocalendar()
#             month = dt.month
#             insert_vals = (timestamp, hour, day, week, month, year, weekday)
#             print(f'insert_vals: {insert_vals}')
#             try:
#                 cur.execute(time_query, insert_vals)
#             except psycopg2.Error as e:
#                     print('caught psycopg2 exception!')
#                     print(e.pgerror)
#                     print(e.diag.message_primary)
#                     continue

In [319]:
# def insert_user_data(users):
#     seen_users = set()
#     for user in users:
#         user_id = user['user_id']
#         if  user_id not in seen_users:
#             seen_users.add(user_id)
#             insert_vals = (user_id, 
#                            user['first'], 
#                            user['last'], 
#                            user['gender'], 
#                            user['level'])
#             try:
#                 cur.execute(user_insert_query, insert_vals)
#             except psycopg2.Error as e:
#                     print('caught psycopg2 exception!')
#                     print(e.pgerror)
#                     print(e.diag.message_primary)
#                     continue

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = 
df.head()

In [ ]:
t = 
t.head()

In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [220]:
df_log = pd.read_json(log_files[0], lines=True)
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [ ]:
# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

In [244]:
df_log = pd.read_json(log_files[1], lines=True)

In [245]:
df_log

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Great Lake Swimmers,Logged In,Kevin,M,0,Arellano,215.11791,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Your Rocky Spine,200,1542931645796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
1,Soziedad Alkoholika,Logged In,Kevin,M,1,Arellano,204.74730,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Va Bien,200,1542931860796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
2,Franz Ferdinand,Logged In,Kevin,M,2,Arellano,172.01587,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Eleanor Put Your Boots On,200,1542932064796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,Modest Mouse,Logged In,Kevin,M,3,Arellano,209.52771,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Float On,200,1542932236796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
4,Adam Lambert,Logged In,Kevin,M,4,Arellano,266.44853,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Aftermath,200,1542932445796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
5,Parni Valjak,Logged In,Kevin,M,5,Arellano,259.83955,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Dok si pored mene,200,1542932711796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
6,None,Logged In,Kevin,M,6,Arellano,NaN,free,"Harrisburg-Carlisle, PA",GET,Home,1.540007e+12,815,None,200,1542932752796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
7,Flaw,Logged In,Kevin,M,7,Arellano,265.89995,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Not Enough,200,1542932970796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
8,Foo Fighters,Logged In,Kevin,M,8,Arellano,250.14812,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Everlong,200,1542933235796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
9,ÃÂngeles del Infierno,Logged In,Kevin,M,9,Arellano,267.25832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,815,Si TÃÂº No EstÃÂ¡s AquÃÂ­,200,1542933485796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


In [246]:
all_songs = []

In [250]:
def get_all_song_titles_from_logs(log_files):
    song_titles = set()
    for file in log_files:
        with open(file, 'r') as f:
            for line in f:
                try:
                    d = json.loads(line)
                except JSONDecodeError as e:
                    print('Msg: {e.msg}, Doc: {e.doc}, Pos: {e.pos}, LineNo: {e.lineno}, ColNo: {e.colno}')
                    continue
                    
                if d['page'] == 'NextSong':
                    song_titles.add(d['song'])
    return list(song_titles)

In [257]:
all_songs_from_logs = get_all_song_titles_from_logs(log_files)

In [258]:
len(all_songs_from_logs)

5189

In [256]:
def get_all_song_titles_from_songs(song_files):
    song_titles = set()
    for file in song_files:
        with open(file, 'r') as f:
            for line in f:
                try:
                    d = json.loads(line)
                except JSONDecodeError as e:
                    print('Msg: {e.msg}, Doc: {e.doc}, Pos: {e.pos}, LineNo: {e.lineno}, ColNo: {e.colno}')
                    continue
                song_titles.add(d['title'])
    return list(song_titles)

In [259]:
all_songs_from_songs = get_all_song_titles_from_songs(song_files)

In [260]:
len(all_songs_from_songs)

71

In [241]:
df_songs.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984


In [269]:
name = "Elena"

In [270]:
title = "Setanta matins"

In [271]:
duration = 269.58322

In [276]:
"SELECT song_id, artist_id FROM songs WHERE title = '{}' and name = '{}' and duration = {}".format(title, name, duration)

"SELECT song_id, artist_id FROM songs WHERE title = 'Setanta matins' and artist_name = 'Elena' and duration = 269.58322"

In [287]:
qry = ("SELECT song_id, s.artist_id \
FROM songs s JOIN artists a on s.artist_id = a.artist_id \
WHERE title = '{}' and a.name='{}' and duration={}".format(title, name, duration))

In [288]:
qry

"SELECT song_id, s.artist_id FROM songs s JOIN artists a on s.artist_id = a.artist_id WHERE title = 'Setanta matins' and a.name='Elena' and duration=269.58322"

In [292]:
cur.execute(qry)

In [293]:
rows = cur.fetchall()
for row in rows:
    print(row)

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.